P1
Load all images and overlay Rectangles onto separate images
  Sub Part: Use Positive grasp rectangle coordinates to generate masks for P3

P2
Map all images using point cloud data (Drop in Chris's P2 Code and loop for all images)

P3
Perform a bunch of channel conversion magic (Chris's Code)
Do Bitwise extraction using stored masks
Plot extracted images

P4
Run extracted images through PCA whitening (Eric's Code)
  May require that we trim the blackspace left from the mask since its pulling averages from each row image pixels.

P5
Visualize Point Cloud Data
  Plot both Raw point cloud data, and data resulting from PCA Analysis per Timi's email  


P1 Load all images and overlay Rectangles onto separate images

  Sub Part: Use Positive grasp rectangle coordinates to generate masks for P3

In [49]:
#Loads/Imports
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import argparse
import glob
import os
import numpy as np
import pandas as pd
from sklearn import decomposition
from sklearn.decomposition import PCA, IncrementalPCA
import torch
import cv2
from PIL import Image
from IPython.display import display
from imageio import imsave

#from image import DepthImage

import tifffile



In [10]:
#P1
# WIP
# Loading Images and Data
# Loop through all image files in the folder, all pngs, tiffs, and masks are all indexed to align

folder_path = "./HW2/data/02/"
files= os.listdir(folder_path) #OS is an operating system object, listdir is a file objkect
files=sorted(files) #sorts in alphabetical order
images=[] #Declare images as an empty object
images_pos=[]
images_neg=[]
neg_rect=[]
pos_rect=[]
masks=[]
RGBDtiff=[]
j=0


for filename in files:
    filename = folder_path + str(filename).strip()
    #print(filename)

    if filename.endswith((".txt")):  #look for png files
        if ("cneg") in filename:
         #print("cneg")
            if os.path.getsize(filename) == 0:
                 print(f"File '{filename}' is empty.")
            else:
                neg_rect= pd.read_csv(filename, delimiter=r'\s+', header=None, engine='python')
                neg_rect = neg_rect.to_numpy()
        elif ("cpos") in filename:
          #print("cpos")
            if os.path.getsize(filename) == 0:
                 print(f"File '{filename}' is empty.")
            else:
                pos_rect= pd.read_csv(filename,delimiter=r'\s+', header=None, engine='python')
                pos_rect = pos_rect.to_numpy()

        else:
          #print("point cloud")
          continue


    elif filename.endswith((".png")):  #look for png files
        #print("png")
        img = mpimg.imread(filename)  # Read the image

        images.append(img)
        img_pos = mpimg.imread(filename)  # Read the image for positive rectangle overlay
        images_pos.append(img_pos)
        img_neg= mpimg.imread(filename)  # Read the image for negative rectangle overlay
        images_neg.append(img_neg)
        mask = np.zeros(images_pos[j].shape[:2], dtype=np.uint8) #Initialize Mask
        for i in range(0,len(pos_rect),4):
          #Plot Rectangles on Images
          rectangle_pts = np.array([[pos_rect[i,0],pos_rect[i,1]], [pos_rect[i+1,0],pos_rect[i+1,1]], [pos_rect[i+2,0],pos_rect[i+2,1]], [pos_rect[i+3,0],pos_rect[i+3,1]]], np.int32)
          rectangle_pts = rectangle_pts.reshape((-1, 1, 2))
          cv2.polylines(images_pos[j], [rectangle_pts], isClosed=True, color=(255, 0, 0), thickness=1)
          #Save Positive Rectangle Masks for use in P3
          polygon = rectangle_pts
          cv2.fillPoly(mask, [polygon], 255)
        masks.append(mask)
        for i in range(0,len(neg_rect),4):
          #Plot Rectangles on Images
          rectangle_pts = np.array([[neg_rect[i,0],neg_rect[i,1]], [neg_rect[i+1,0],neg_rect[i+1,1]], [neg_rect[i+2,0],neg_rect[i+2,1]], [neg_rect[i+3,0],neg_rect[i+3,1]]], np.int32)
          rectangle_pts = rectangle_pts.reshape((-1, 1, 2))
          cv2.polylines(images_neg[j], [rectangle_pts], isClosed=True, color=(255, 0, 0), thickness=1)
        j=j+1
        #print(j)
        continue

    elif filename.endswith(("_RGBD.tiff")):  #look for tiff files, Chris's Code here for loading tiff files
        #print("")
        #img_path = os.path.join(folder_path, filename)
        #print("tiff")
        tiffin = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        RGBDtiff.append(tiffin)
        continue

print("Loaded", len(images), "images")




Loaded 99 images


In [7]:
print(len(depth))
print(len(masks))

99
99


In [ ]:
#P1 Inspect Dataset (when changing indices ensure they all match)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(images[0])
plt.show()
plt.figure(figsize=(12, 6))
plt.imshow(images_pos[0])
plt.show()
plt.figure(figsize=(12, 6))
plt.imshow(images_neg[0])
plt.show()
plt.figure(figsize=(12, 6))
plt.imshow(masks[0])
plt.show()
plt.figure(figsize=(12, 6))
#Trim Image using Mask
extracted = cv2.bitwise_and(images[0], images[0], mask=masks[0])
plt.imshow(extracted)
plt.show()
plt.figure(figsize=(12, 6))
plt.imshow(RGBDtiff[0])


P2 Map all images using point cloud data (Drop in Chris's P2 Code and loop for all images)

In [ ]:
#P2



## From image.py it calls the DepthImage class and creates RGB and Depth Images

Note: The class "DepthImage" was modified so that it created RGB and Depth images.
Original Code Source: https://github.com/skumra/robotic-grasping/blob/master/utils/dataset_processing/image.py

In [ ]:
#P2  No need to call this again since the new RGB and Depth files have been created
"""
path = './data/01/'
pcds = glob.glob(os.path.join(path, 'pcd[0-9][0-9][0-9][0-9].txt'))  # Matches 'pcdXXXX.txt'
pcds.sort()

for pcd in pcds:
    di, rgb_img = DepthImage.from_pcd(pcd, (480, 640))

    # Save depth image
    depth_of_name = pcd.replace('.txt', '_depth.tiff')
    imwrite(depth_of_name, di.img.astype(np.float32))

    # Save RGB image
    rgb_of_name = pcd.replace('.txt', '_rgb.png')
    imwrite(rgb_of_name, rgb_img)
    """

P3
Perform a bunch of channel conversion magic (Chris's Code)
Do Bitwise extraction using stored masks
Plot extracted images

In [ ]:
#P3/ P4

#####################YUV Conversion


##############Grasp Rectangle Extraction using Masks
extracts=[]
cropped_images = []
image_recon = []
# Load the saved 4-channel RGB-D image
#Reference tiff array for this
i=0
for items in RGBDtiff:
  rgbd_new = RGBDtiff[i]

  # Extract RGB and Depth for visualization
  rgb_new = rgbd_new[:, :, :3]
  depth_new = rgbd_new[:, :, 3]

  #Sub-Patch Extraction: Use Masks saved from P1
  extracted = cv2.bitwise_and(depth_new, depth_new, mask=masks[i])
  extracts.append(extracted)
  #Contour Cleanup
  #Trimming and Visualization

  #Trim Image with Mask Contours to limit Mask effect on PCA

  contours, _ = cv2.findContours(masks[i], cv2.RETR_LIST , cv2.CHAIN_APPROX_NONE)

  # Grab the largest contour is the ROI (lossy, does not always get all grasp rectangles in cropped image)
  c = max(contours, key=cv2.contourArea)

  # Get the bounding box coordinates
  x, y, w, h = cv2.boundingRect(c)
  #Crop Image
  crop= extracted[y:y+h, x:x+w]
  cropped_images.append(crop)

  #PCA Ops
  pca = PCA()
  pca.fit(cropped_images[i])
  # Getting the cumulative variance
  var_cumu = np.cumsum(pca.explained_variance_ratio_)*100

  # How many PCs explain 99% of the variance?
  k = np.argmax(var_cumu>99)

  #Perform PCA and generate reconstructed images
  ipca = IncrementalPCA(whiten=True, n_components=k) #set value to k (99% variance) so process can be looped
  recon = ipca.inverse_transform(ipca.fit_transform(cropped_images[i]))
  image_recon.append(recon)

  i=i+1

In [ ]:
  #P3/P4 Inspect Output Single Image
  #THis is for actually cutting the images with the masks
  plt.figure(figsize=(12, 6))
  plt.imshow(images[97])
  plt.figure(figsize=(12, 6))
  plt.imshow(masks[97])

  plt.figure(figsize=(12, 6))
  plt.imshow((extracts[97]), cmap='jet', vmin=0, vmax=0.1)
  plt.colorbar()
  plt.title("Depth Image with Grasping Rectangles")
  plt.figure(figsize=(12, 6))
  plt.imshow((cropped_images[97]), cmap='jet', vmin=0, vmax=0.1)
  plt.colorbar()
  plt.title("Cropped Depth Image with Grasping Rectangles")
  #display(Image.fromarray(roi))

In [ ]:
#P3/P4 #Data Inspection, used for generating video, LONG RUNTIME ON JS2 (90s)
  #print("New RGB-D Shape:", rgbd_new.shape)  # Expected output: (480, 640, 4)

  # Extract RGB and Depth for visualization
# i=0
# for items in RGBDtiff:
#   rgbd_new = RGBDtiff[i]
#   rgb_new = rgbd_new[:, :, :3]
#   depth_new = rgbd_new[:, :, 3]
#   plt.figure(figsize=(12, 6))
#   plt.imshow(extracts[i])
#   plt.figure(figsize=(12, 6))
#   plt.figure(figsize=(15, 5))
#   plt.subplot(1, 3, 1)
#   plt.imshow(cropped_images[i] , cmap='jet', vmin=0, vmax=0.1)
#   plt.colorbar()
#   plt.title("Depth Image")
#   plt.subplot(1, 3, 2)
#   plt.imshow((image_recon[i]) ,cmap='jet', vmin=0, vmax=0.1)
#   plt.colorbar()
#   plt.title("Reconstructed Depth Image")
#   i=i+1


In [ ]:
#P5
#Point Cloud Visualization